#### 1. 환경 설정

* ctrl + shift + p -> Python: Restart Language Server 를 선택하면 노란 warining 줄이 사라집니다.

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print(TAVILY_API_KEY[:4])

gs
Fy
tvly


`(2) 라이브러리`

In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

## 2. 도구 호출 (Tool Calling)
- 도구 호출은 LLM이 특정 작업을 수행하기 위해 외부 기능을 호출하는 기능
- 이를 통해 LLM은 외부 API 통합 등 더 복잡한 작업을 수행할 수 있음 

### 2-1. 랭체인 내장 도구
- Tavily 웹 검색 도구 (예시)

`(1) 도구(tool) 정의하기`

In [4]:
from langchain_community.tools import TavilySearchResults

# 검색할 쿼리 설정
query = "스테이크와 어울리는 와인을 추천해주세요."

# Tavily 검색 도구 초기화 (최대 3개의 결과 반환)
#web_search = TavilySearchResults(max_results=2)
web_search = TavilySearchResults(
    # 검색 결과의 최대 개수를 3개로 제한합니다.
    max_results=3,

    # 검색 깊이를 "고급(advanced)"으로 설정합니다.
    # 일반(basic) 검색보다 더 깊이 있고 포괄적인 결과를 제공해 복잡한 질문에 더 적합합니다.
    search_depth="advanced",

    # 검색 결과에 질문에 대한 요약된 답변을 포함시킵니다.
    # 이를 통해 에이전트가 별도의 추론 없이 핵심 정보를 빠르게 얻을 수 있습니다.
    include_answer=True,

    # 검색 결과에 원본 웹페이지의 전체 콘텐츠를 포함시킵니다.
    # 답변에 대한 더 자세한 맥락이나 원문이 필요할 때 유용합니다.
    include_raw_content=True,

    # 검색 결과에 관련된 이미지를 포함시킵니다.
    # 시각적인 정보가 필요한 경우(예: 특정 장소, 제품 등) 유용합니다.
    include_images=True,
)

# 웹 검색 실행
search_results = web_search.invoke(query)

# 검색 결과 출력
for result in search_results:
    print(type(result))
    print(result)  
    print("-" * 100)  

<class 'dict'>
{'title': '스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? - 마시자 매거진', 'url': 'https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/', 'content': '# 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가?\n\n카베르네 소비뇽(Cabernet Sauvignon) 및 말벡(Malbec)과 같은 전형적인 선택부터 더 가벼운 레드 와인, 심지어 화이트 와인과 맛있는 스테이크를 페어링하는 방법까지, 우리의 아카이브에서 가져온 최고의 조언과 최근 디캔터 전문가가 추천한 와인을 소개한다.\n\n<스테이크를 곁들인 레드 와인을 위한 5가지 전형적인 선택>\n\n• 카베르네 소비뇽(Cabernet Sauvignon)  \n• 말벡(Malbec)  \n• 그르나슈/쉬라즈 블렌드(Grenache / Shiraz blends)  \n• 시라/쉬라즈(Syrah / Shiraz)  \n• 산지오베제(Sangiovese)\n\n육즙이 풍부한 스테이크와 맛있는 와인이 있는 저녁 식사는 적어도 고기 애호가들에게 인생의 큰 즐거움일 것이다. [...] ‘멋지고 생동감 넘치는 카베르네 프랑(Cabernet Franc)은 어떤가? 아니면 카리냥(Carignan), 쌩소(Cinsault) 또는 서늘한 기후에서 생산한 시라(Syrah)는 어떨까? DWWA 칠레 지역 의장이자 Decanter Retailer Awards 회장인 리차즈는 “풀바디하지만 우아한 로제(rosé)도 따뜻한 날에는 잘 어울린다.”라고 말했다.\n\n그는 바디감과 질감이 있지만 스테이크 저녁 식사 중에 미각을 상쾌하게 할 수 있는

In [ ]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

print("schema: ")
pprint(web_search.args_schema.schema())
print("-"*100)

`(2) 도구(tool) 호출하기`

In [ ]:
# from langchain_openai import ChatOpenAI
# # # ChatOpenAI 모델 초기화
# llm = ChatOpenAI(model="gpt-4o-mini")

from langchain_upstage import ChatUpstage
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print(llm)

# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     #model="meta-llama/llama-4-scout-17b-16e-instruct",
#     model="moonshotai/kimi-k2-instruct-0905",
#     temperature=0
# )

# 웹 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[web_search])
print(type(llm_with_tools))

In [ ]:
# 도구 호출이 필요 없는 LLM 호출을 수행
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [ ]:
from pprint import pprint

# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_message = llm_with_tools.invoke(query)
print(type(ai_message))

# AIMessage의 속성 확인
pprint(vars(ai_message))

In [ ]:

pprint(ai_message)
print("#" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_message.content)
print("*" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_message.tool_calls)
print("-" * 100)

In [ ]:
tool_call = ai_message.tool_calls[0]

pprint(tool_call)

`(3) 도구(tool) 실행하기`

##### Tool 직접 호출하여 바로 ToolMessage 객체 생성

* 이 방법은 도구를 직접 호출하여 ToolMessage 객체를 생성한다.
* 가장 간단하고 직관적인 방법으로, LangChain의 추상화를 활용한다.

In [ ]:
# tool_call  {'name': 'tavily_search_results_json', 'args': {'query': 'wine pairing with steak'}, 'id': 'call_LrHyxTadqHDjW7J6LOWEaoSi', 'type': 'tool_call'}
tool_message = web_search.invoke(tool_call)
print(type(tool_message))

# 특정 속성들만 확인
print("\n=== 주요 속성들 ===")
attributes = ['content', 'tool_call_id', 'name', 'type', 'additional_kwargs']
for attr in attributes:
    if hasattr(tool_message, attr):
        print(f"{attr}: {getattr(tool_message, attr)}")


In [ ]:
pprint(tool_message.content)

In [ ]:
print(tool_message.tool_call_id)

In [ ]:
print(tool_message.name)

In [ ]:
ai_message.tool_calls

In [ ]:
# batch 실행 - 도구 호출이 여러 개인 경우

# tool_messages = web_search.batch([tool_call])

tool_messages = web_search.batch(ai_message.tool_calls)

print(tool_messages)
print("-" * 100)
pprint(tool_messages[0].content)

`(4) @chain 데코레이터를 사용하여 RunnableLambda로 생성하기`
* @chain 데코레이터를 사용하여 Tavily를 호출하는 TavilySearchResults 객체와 LLM을 연결하여 RunnableLambda 객체로 생성하기

* @chain 데코레이터를 선언한 web_search_chain 함수의 파라미터 이름이 config여야 하는 이유: 
    * @chain 데코레이터가 config라는 이름만 특별히 인식하도록 설계되었습니다.
    * config 생략 가능한 이유: @chain이 자동으로 기본 RunnableConfig()를 생성해서 전달하기 때문입니다.

In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from pprint import pprint

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
#llm = ChatOpenAI(model="gpt-4o-mini")

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
tavily_tool = TavilySearchResults(max_results=2)

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_tool])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
# web_search_chain() 함수는 StructuredTool 객체이다.
@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msgs = llm_chain.invoke(input_, config=config)
    print("ai_msgs: \n", ai_msgs)
    print("-"*100)

    tool_msgs = tavily_tool.batch(ai_msgs.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msgs, *tool_msgs]}, config=config)

# 체인 실행
query = "스테이크와 어울리는 와인을 추천해주세요."
# chain을 호출하는 함수 호출
print('====>' , type(web_search_chain))
response = web_search_chain.invoke(query)

# 응답 출력 
pprint(response.content)

### 2-2. 사용자 정의 도구
* @tool decorator를 통해 사용자 정의 도구를 정의할 수 있음
* @tool 데코레이터를 사용하여 Tavily를 호출하는 tavily_search_func 함수를 StructuredTool로 생성하기

`(1) 도구(tool) 정의하기`

In [ ]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# Tool 정의 
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

### 2-3. Runnable 객체를 도구(tool) 변환
- 문자열이나 dict 입력을 받는 Runnable을 도구로 변환
- as_tool() 메서드를 사용
- [WikipediaLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.wikipedia.WikipediaLoader.html) 사용


- wikipedia 설치
```python
    poetry add wikipedia
```

In [ ]:
# 필요한 모듈들을 가져옵니다.
from langchain_community.document_loaders import WikipediaLoader  # 위키피디아에서 문서를 불러오는 로더
from langchain_core.documents import Document                     # LangChain 문서 객체
from langchain_core.runnables import RunnableLambda             # 일반 함수를 LangChain의 Runnable로 변환
from pydantic import BaseModel, Field                           # 데이터 유효성 검사 및 스키마 정의를 위한 Pydantic
from typing import List                                         # 타입 힌트를 위한 List
# dedent() 함수를 가져옵니다. 이 함수는 문자열의 들여쓰기를 제거하는 역할을 합니다.
from textwrap import dedent

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하는 함수를 정의합니다.
def search_wiki(input_data: dict) -> List[Document]:
    """사용자의 입력(query)을 기반으로 위키피디아 문서를 검색하고 지정된 개수(k)만큼 반환합니다."""
    # input_data 딕셔너리에서 'query' 값을 가져와 검색어로 사용합니다.
    query = input_data["query"]
    # input_data에서 'k' 값을 가져옵니다. 'k'가 없으면 기본값 2를 사용합니다.
    k = input_data.get("k", 2)
    
    # 지정된 검색어와 문서 개수(k), 언어('ko' for 한국어)를 설정하여 WikipediaLoader를 초기화합니다.
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    # 로더를 실행하여 위키피디아 문서를 불러옵니다.
    wiki_docs = wiki_loader.load()
    # 불러온 문서 목록을 반환합니다.
    return wiki_docs

# 도구 호출에 사용될 입력 스키마(데이터 구조)를 Pydantic을 이용해 정의합니다.
class WikiSearchSchema(BaseModel):
    """위키피디아 검색을 위한 입력 스키마."""
    # 검색어를 나타내는 필드. ...은 필수 필드임을 의미합니다.
    query: str = Field(..., description="위키피디아에서 검색할 질의어")
    # 반환할 문서의 개수를 나타내는 필드. 기본값은 2입니다.
    k: int = Field(2, description="반환할 문서의 개수 (기본값은 2)")

# RunnableLambda를 사용하여 함수(search_wiki)를 LangChain의 Runnable 객체로 변환합니다.
# 이를 통해 함수를 LangChain의 체인(chain)이나 에이전트 도구로 활용할 수 있습니다.
runnable = RunnableLambda(search_wiki)

# Runnable 객체를 도구(tool)로 변환합니다. wiki_search는 StructuredTool 객체이다.
wiki_search = runnable.as_tool(
    # 도구의 이름을 'wiki_search'로 설정합니다.
    name="wiki_search",
    # 도구의 설명을 지정합니다. dedent() 함수를 사용해 들여쓰기를 제거하여 깔끔하게 만듭니다.
    description=dedent("""
        위키피디아 정보를 검색해야 할 때 이 도구를 사용하세요.
        사용자의 질의와 관련된 위키피디아 문서를 검색하여 지정된 개수만큼 반환합니다.
        일반적인 지식이나 배경 정보가 필요할 때 유용합니다.
    """),
    # 도구의 입력 스키마를 WikiSearchSchema로 지정하여 입력의 유효성을 검사합니다.
    args_schema=WikiSearchSchema
)
print(type(wiki_search))

In [ ]:
# 도구 속성
print("자료형: ")
print(type(wiki_search))
print("-"*100)

print("name: ")
print(wiki_search.name)
print("-"*100)

print("description: ")
pprint(wiki_search.description)
print("-"*100)

print("schema: ")
pprint(wiki_search.args_schema.schema())
print("-"*100)

In [ ]:
# 위키 검색 Tool 호출
query = "파스타의 유래"
wiki_results = wiki_search.invoke({"query":query})

# 검색 결과 출력
for result in wiki_results:
    print(result)  
    print("-" * 100)  

In [ ]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
print(llm)
print('==> tavily_search_func = ' , type(tavily_search_func))
print('==> wiki_search = ' , type(wiki_search))

llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_search])
print(type(llm_with_tools))

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msgs = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msgs)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msgs.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msgs.tool_calls)
print("-" * 100)

`(2) LCEL 체인`
- 위키피디아 문서를 검색하고 내용을 요약하는 체인

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"파스타의 유래"})
pprint(summarized_text)

In [ ]:
from pydantic import BaseModel, Field                           # 데이터 유효성 검사 및 스키마 정의를 위한 Pydantic
from typing import List                                         # 타입 힌트를 위한 List
# dedent() 함수를 가져옵니다. 이 함수는 문자열의 들여쓰기를 제거하는 역할을 합니다.
from textwrap import dedent

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

# 도구 속성
print("자료형: ")
print(type(wiki_summary))
print("1","-"*100)

print("name: ")
print(wiki_summary.name)
print("2","-"*100)

print("description: ")
pprint(wiki_summary.description)
print("3","-"*100)

print("schema: ")
pprint(wiki_summary.args_schema.schema())
print("4","-"*100)

In [ ]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_summary])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msgs = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msgs)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msgs.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msgs.tool_calls)
print("-" * 100)

In [ ]:
ai_msgs.tool_calls[1]

In [ ]:
# 도구 실행 
tool_message = wiki_summary.invoke(ai_msgs.tool_calls[1])

print(tool_message)
print("-" * 100)
pprint(tool_message.content)

In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_summary])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def wiki_summary_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = wiki_summary.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = wiki_summary_chain.invoke("파스타의 유래에 대해서 알려주세요.")

# 응답 출력 
pprint(response.content)

### 2-4. 벡터저장소 검색기
- @tool decorator 사용

`(1) 문서 로드 및 인덱싱`

In [ ]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("../data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

In [ ]:
# 문서 분할 (Chunking)
# 이 함수는 긴 문서(document)를 더 작은 단위인 '메뉴 항목(menu_items)'으로 분할합니다.
# LangChain의 RAG(Retrieval-Augmented Generation) 시스템에서 정확한 정보를 찾기 위해
# 문서를 의미 있는 청크(chunk)로 나누는 과정입니다.
import re
from langchain_core.documents import Document

def split_menu_items(document):
    """
    하나의 큰 문서(document)에서 각 메뉴 항목을 분리하여
    각각을 별도의 Document 객체로 반환하는 함수입니다.
    """
    
    # 정규표현식(regular expression)을 정의합니다. 
    # 이 패턴은 "숫자. "로 시작하고, 다음 "숫자. " 또는 문서의 끝이 나타날 때까지의 텍스트를 찾습니다.
    # r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    #   - r'...' : raw string으로, \ 문자를 그대로 인식하게 합니다.
    #   - (\d+\.\s.*?) : 캡처 그룹(괄호).
    #     - \d+ : 하나 이상의 숫자.
    #     - \. : 마침표(.) 문자.
    #     - \s : 공백 문자.
    #     - .*? : 최소 일치(non-greedy)로 어떤 문자든 0개 이상.
    #   - (?=\n\n\d+\.|$) : 전방탐색(lookahead).
    #     - 다음 패턴이 뒤따르는 위치를 찾지만, 실제 매치 결과에는 포함하지 않습니다.
    #     - \n\n\d+\. : 두 개의 줄바꿈과 "숫자. "가 이어지는 부분.
    #     - | : '또는'을 의미합니다.
    #     - $ : 문자열의 끝.
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    
    # document의 page_content(문서 내용)에서 위에서 정의한 패턴과 일치하는 모든 문자열을 찾아 리스트로 반환합니다.
    # re.DOTALL 플래그는 .이 줄바꿈 문자(\n)까지 매치하도록 합니다.
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 분리된 각 메뉴 항목을 저장할 빈 리스트를 만듭니다.
    menu_documents = []
    
    # enumerate()를 사용하여 각 메뉴 항목과 그 인덱스(i)를 순회합니다. 인덱스는 1부터 시작합니다.
    for i, item in enumerate(menu_items, 1):
        # 각 메뉴 항목 텍스트에서 첫 번째 줄을 가져와 '숫자.' 부분을 제거하고 메뉴 이름을 추출합니다.
        # 예: "1. 불고기 덮밥\n가격..." -> "불고기 덮밥"
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 LangChain Document 객체를 생성합니다.
        menu_doc = Document(
            # page_content에는 현재 메뉴 항목의 전체 텍스트를 할당합니다.
            page_content=item.strip(),
            # 문서와 관련된 메타데이터(부가 정보)를 딕셔너리 형태로 저장합니다.
            metadata={
                # 원본 문서의 출처(파일 경로 등)를 그대로 가져옵니다.
                "source": document.metadata['source'],
                # 메뉴의 순번을 저장합니다.
                "menu_number": i,
                # 추출한 메뉴 이름을 저장합니다.
                "menu_name": menu_name
            }
        )
        
        # 새로 생성한 Document 객체를 리스트에 추가합니다.
        menu_documents.append(menu_doc)
    
    # 모든 메뉴 항목이 Document 객체로 변환된 리스트를 반환합니다.
    return menu_documents

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

### Embedding
* OllamaEmbeddings
    * ollama run qwen2.5:1.5b
* UpstageEmbeddings
    * solar-embedding-1-large

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_upstage import UpstageEmbeddings

#embeddings_model = OllamaEmbeddings(model="qwen2.5:1.5b") 
embeddings_model = UpstageEmbeddings(model="solar-embedding-1-large")

# FAISS 인덱스 생성
menu_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
menu_db.save_local("../db/menu_db")


# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 4},
)

# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

- 와인 메뉴에 대해서도 같은 작업을 처리

In [ ]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("../data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")


# FAISS 인덱스 생성
wine_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# 병합된 DB 다시 저장
wine_db.save_local("../db/wine_db")

wine_retriever = menu_db.as_retriever(
    search_kwargs={'k': 4},
)

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

`(2) 도구(tool) 정의하기`

In [ ]:
from langchain_core.tools import tool
# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "../db/menu_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_menu_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=4)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_menu_func))
print("-"*100)

print("name: ")
print(db_search_menu_func.name)
print("-"*100)

print("description: ")
pprint(db_search_menu_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_menu_func.args_schema.schema())
print("-"*100)

In [ ]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# wine db 벡터 저장소 로드
wine_db = FAISS.load_local(
    "../db/wine_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_wine_func(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs
   
   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_wine_func))
print("-"*100)

print("name: ")
print(db_search_wine_func.name)
print("-"*100)

print("description: ")
pprint(db_search_wine_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_wine_func.args_schema.schema())
print("-"*100)

In [ ]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[db_search_menu_func, db_search_wine_func])

# 도구 호출이 필요한 LLM 호출을 수행
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 와인도 추천 해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

`(3) 여러 개의 도구(tool) 호출하기`

In [ ]:
tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
for tool in tools:
    print(tool.name)

In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
#llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_wine_func":
            tool_message = db_search_wine_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 와인도 추천해주세요.")

# 응답 출력 
print(response.content)

In [ ]:
# 체인 실행
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 파스타의 역사 또는 유래를 알려주시고 서울 강남의 파스타 맛집도 추천해주세요.")

# 응답 출력 
print(response.content)

## 3. Few-shot 프롬프팅 
- 각 도구의 용도를 구분하여 few-shot 예제로 제시

### 3-1. Few-shot 도구 호출

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_menu_func", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),    
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. \
                주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_wine_func", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. \
                2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다.\
               주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, \
              주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. \
              특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오,\
               그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
]

system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
#llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인도 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

In [ ]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 가장 최근 파스타 맛집을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

### 3-2. 답변 생성 체인 

In [ ]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
#llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        # [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_wine_func":
            tool_message = db_search_wine_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해 주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

In [ ]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 가장 최근 파스타 맛집을 추천해주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

## 4. LangChain Agent 사용
- 유의사항: 프롬프트에 "agent_scratchpad",  "input" 변수를 포함

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        You are an AI assistant providing restaurant menu information and general food-related knowledge. 
        Your main goal is to provide accurate information and effective recommendations to users.

        Key guidelines:
        1. For restaurant menu information, use the search_menu tool. This tool provides details on menu items, including prices, ingredients, and cooking methods.
        2. For general food information, history, and cultural background, utilize the wiki_summary tool.
        3. For wine recommendations or food and wine pairing information, use the search_wine tool.
        4. If additional web searches are needed or for the most up-to-date information, use the search_web tool.
        5. Provide clear and concise responses based on the search results.
        6. If a question is ambiguous or lacks necessary information, politely ask for clarification.
        7. Always maintain a helpful and professional tone.
        8. When providing menu information, describe in the order of price, main ingredients, and distinctive cooking methods.
        9. When making recommendations, briefly explain the reasons.
        10. Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.


        Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
        Combine the tools to provide the most comprehensive and accurate answers to user queries. 
        Always strive to provide the most current and accurate information.
        """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [ ]:
# Tool calling Agent 생성
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# AgentExecutor 생성 
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# AgentExecutor 실행

query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
agent_response = agent_executor.invoke({"input": query})

In [ ]:
pprint(agent_response)

## 5. Gradio 활용

In [ ]:
import gradio as gr
from typing import List, Tuple

def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))
        
        # agent_executor를 사용하여 응답 생성
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history[-2:]    # 최근 2개의 메시지 기록만을 활용 
        })
        
        # agent_executor의 응답에서 최종 답변 추출
        return response['output']
    except Exception as e:
        # 오류 발생 시 사용자에게 알리고 로그 기록
        print(f"Error occurred: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."

# 예제 질문 정의
example_questions = [
    "시그니처 스테이크의 가격과 특징을 알려주세요.",
    "트러플 리조또와 잘 어울리는 와인을 추천해주세요.",
    "해산물 파스타의 주요 재료는 무엇인가요? 서울 강남 지역에 레스토랑을 추천해주세요.",
    "채식주의자를 위한 메뉴 옵션이 있나요?"
]

# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="레스토랑 메뉴 AI 어시스턴트",
    description="메뉴 정보, 추천, 음식 관련 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft()
)

# 데모 실행
demo.launch()

In [ ]:
# 데모 종료
demo.close()